# Proceso como objeto
En el siguiente ejemplo se muestra como crear un proceso por medio de la definición de la clase `MyProcess` que hereda de la clase base `multiprocessing.Process`. La clase creada incluye un constructor y un método `run`.

In [1]:
import multiprocessing

class MyProcess(multiprocessing.Process):
    def __init__(self):
        super(MyProcess, self).__init__()

    def run(self):
        print("Child process PID: {}".format(multiprocessing.current_process().pid))

if __name__ == '__main__':
    print("Main process PID: {}".format(multiprocessing.current_process().pid))
    my_process = MyProcess()
    my_process.start()
    my_process.join()

Main process PID: 30119
Child process PID: 30216


## Comunicación entre procesos
Consideremos el siguiente ejemplo donde el proceso padre quiere consultar el resultado de una operación que ejecutó el proceso hijo: 

In [2]:
import multiprocessing as mp
import time
import os

In [3]:
num_res = []

def calc_cuad(numeros):
    global num_res
    for n in numeros:
        print("cuadrado: ", n*n)
        num_res.append(n * n)

nums = range(10)
t = time.time()
p1 = mp.Process(target=calc_cuad, args=(nums, ))
p1.start()
p1.join()

print("Tiempo de ejecución: ", time.time()-t)
print("Resultado operación: ", num_res)
print("Finaliza ejecución")


cuadrado:  0
cuadrado:  1
cuadrado:  4
cuadrado:  9
cuadrado:  16
cuadrado:  25
cuadrado:  36
cuadrado:  49
cuadrado:  64
cuadrado:  81
Tiempo de ejecución:  0.09531569480895996
Resultado operación:  []
Finaliza ejecución


Los procesos tienen su propio espacio de memoria. Así, las variables del programa no se comparten entre procesos. Es necesario crear comunicación entre procesos (IPC) si se desea compoartir variables. El módulo `multiprocessing` permite la comunicación por medio de pipes y colas. 

## Memoria Compartida
Cuando se realiza una programación concurrente, generalmente es mejor evitar el uso de recursos compartidos en la medida de lo posible. Esto es cierto cuando se utilizan múltiples procesos. Sin embargo, si se necesita utilizar algunos datos compartidos, el multiprocesamiento proporciona los medios necesarios para hacerlo. En memoria caché se guarda la memoria de cada proceso, es decir, cada proceso tendrá su memoria caché. 

Memoria: 
- $P1$:
    - registros
    - mem caché 

- $P2$:
    - registros
    - mem caché 

- $P3$:
    - registros
    - mem caché 

In [5]:
nums = [3, 4, 5]
for idx, i in enumerate(nums):
    print(idx, i)

0 3
1 4
2 5


In [6]:
nums = range(10)
for idx, i in enumerate(nums):
    print(idx, i)

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9


Se pueden compartir los datos del proceso hijo al proceso padre por medio del método `Array` de `multiprocessing`. 

In [11]:
def calc_cuad(numeros, result):
    for idx, n in enumerate(numeros):
        result[idx] = n*n
    print("Resultado del proceso: ", result[:])

nums = range(10)
t = time.time()
result = mp.Array('i', 10)
p1 = mp.Process(target=calc_cuad, args=(nums, result))

p1.start()
p1.join()

print("Resultado fuera del proceso", result[:])
print("Tiempo de ejecución: ", time.time() - t)
print("Finaliza la ejecución")

Resultado del proceso:  [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
Resultado fuera del proceso [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
Tiempo de ejecución:  0.021578550338745117
Finaliza la ejecución


También es posible compartir valores unitarios del proceso hijo de otra manera: `Value`

In [13]:
def calc_cuad(numeros, result, val):
    val.value = 5.35
    for idx, n in enumerate(numeros):
        result[idx] = n*n
    print("Resultado del proceso: ", result[:])

nums = range(10)
t = time.time()
result = mp.Array('i', 10)
val = mp.Value('d', 0.0)
p1 = mp.Process(target=calc_cuad, args=(nums, result, val))

p1.start()
p1.join()

print("Resultado fuera del proceso", result[:])
print("Resultado fuera del proceso", val.value)
print("Tiempo de ejecución: ", time.time() - t)
print("Finaliza la ejecución")

Resultado del proceso:  [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
Resultado fuera del proceso [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
Resultado fuera del proceso 5.35
Tiempo de ejecución:  0.02597665786743164
Finaliza la ejecución


## Pipes    


In [14]:
from multiprocessing import Process, Pipe

def f(conn):
    conn.send(["hello world"])
    conn.close()

if __name__ == "__main__":
    parent_conn, child_conn = Pipe()
    p = Process(target=f, args=(child_conn, ))
    p.start()
    print(parent_conn.recv())
    p.join()

['hello world']


### Doble comunicación

In [24]:
from multiprocessing import Process, Pipe
def worker(conn):
    print(conn.recv())
    conn.send("Sent from the child process")
    conn.close() 

conn1, conn2 = Pipe()
process = Process(target=worker, args=(conn2, ))
process.start()

conn1.send("Sent from the main process")
print(conn1.recv())
process.join()

Sent from the main process
Sent from the child process


Ejercicio: extiende el código para que dos procesos reciban un mensaje del padre y cada uno devuelva un mensaje al padre. 

In [26]:
from multiprocessing import Process, Pipe

nombres = ["Carlos", "Renata", "Rebeca", "Sandra", "END"]

def send_msgs(conn, msgs):
    for msg in msgs:
        conn.send(msg)
    conn.close()

def recv_msgs(conn):
    while 1:
        msg = conn.recv()
        if msg == "END":
            break
        print(msg)

parent_conn, child_conn = Pipe()
p1 = Process(target=send_msgs, args=(parent_conn, nombres))
p2 = Process(target=recv_msgs, args=(child_conn,))
p1.start()
p2.start()
p1.join()
p2.join()

Carlos
Renata
Rebeca
Sandra


## Colas


In [29]:
from multiprocessing import Process, Queue

def f(q):
    q.put([42, None, 'hello'])

if __name__ == '__main__':
    q = Queue()
    p = Process(target=f, args=(q, ))
    p.start()
    print(q.get())
    p.join()

[42, None, 'hello']


In [31]:
import multiprocessing

def calc_cuad(numeros, q):
    for n in numeros:
        q.put(n * n)

if __name__ == "__main__":
    nums = range(10)
    q = multiprocessing.Queue()
    p = multiprocessing.Process(target=calc_cuad, args=(nums, q))
    p.start()
    p.join()
    while q.empty() is False:
        print(q.get())

0
1
4
9
16
25
36
49
64
81
